## CSCI 470 Activities and Case Studies

1. For all activities, you are allowed to collaborate with a partner. 
1. For case studies, you should work individually and are **not** allowed to collaborate.

By filling out this notebook and submitting it, you acknowledge that you are aware of the above policies and are agreeing to comply with them.

Some considerations with regard to how these notebooks will be graded:

1. You can add more notebook cells or edit existing notebook cells other than "# YOUR CODE HERE" to test out or debug your code. We actually highly recommend you do so to gain a better understanding of what is happening. However, during grading, **these changes are ignored**. 
2. You must ensure that all your code for the particular task is available in the cells that say "# YOUR CODE HERE"
3. Every cell that says "# YOUR CODE HERE" is followed by a "raise NotImplementedError". You need to remove that line. During grading, if an error occurs then you will not receive points for your work in that section.
4. If your code passes the "assert" statements, then no output will result. If your code fails the "assert" statements, you will get an "AssertionError". Getting an assertion error means you will not receive points for that particular task.
5. If you edit the "assert" statements to make your code pass, they will still fail when they are graded since the "assert" statements will revert to the original. Make sure you don't edit the assert statements.
6. We may sometimes have "hidden" tests for grading. This means that passing the visible "assert" statements is not sufficient. The "assert" statements are there as a guide but you need to make sure you understand what you're required to do and ensure that you are doing it correctly. Passing the visible tests is necessary but not sufficient to get the grade for that cell.
7. When you are asked to define a function, make sure you **don't** use any variables outside of the parameters passed to the function. You can think of the parameters being passed to the function as a hint. Make sure you're using all of those variables.
8. Finally, **make sure you run "Kernel > Restart and Run All"** and pass all the asserts before submitting. If you don't restart the kernel, there may be some code that you ran and deleted that is still being used and that was why your asserts were passing.

# Deep Learning - Autoencoders

In this exercise we'll use an AutoEncoder to learn a dimenionally reduced representation of data and investigate its performance compared to using the original data. You'll learn how to build AutoEncoders and how to use the keras functional API.

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model, Input
import sklearn as sk
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import numpy as np


np.random.seed(0)
tf.random.set_seed(0)

In [2]:
data = load_breast_cancer()
features = data["data"]
targets = data["target"]
X_train, X_test, y_train, y_test = train_test_split(features, targets, random_state=0)

In [3]:
# Read through the description of the data to better understand it
# What features do we have and what is the target we're trying to predict?
print(data["DESCR"])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [4]:
X_train.shape

(426, 30)

In [5]:
X_train

array([[1.185e+01, 1.746e+01, 7.554e+01, ..., 9.140e-02, 3.101e-01,
        7.007e-02],
       [1.122e+01, 1.986e+01, 7.194e+01, ..., 2.022e-02, 3.292e-01,
        6.522e-02],
       [2.013e+01, 2.825e+01, 1.312e+02, ..., 1.628e-01, 2.572e-01,
        6.637e-02],
       ...,
       [9.436e+00, 1.832e+01, 5.982e+01, ..., 5.052e-02, 2.454e-01,
        8.136e-02],
       [9.720e+00, 1.822e+01, 6.073e+01, ..., 0.000e+00, 1.909e-01,
        6.559e-02],
       [1.151e+01, 2.393e+01, 7.452e+01, ..., 9.653e-02, 2.112e-01,
        8.732e-02]])

In [6]:
y_train.shape

(426,)

In [7]:
y_train

array([1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In this exercise, instead of using the sequential model, we will use the base Model in tf.keras. There are two approaches to use tf.keras.Model and we will use the functional api as outlined in the [Model docs](https://www.tensorflow.org/api_docs/python/tf/keras/models/Model).
Note that unlike the manner in which we defined our model in the prior Activity, in this Activity (using the base Model) the definition is more like that of the coding of a functional algorithm, e.g.:

b = f1(a)

c = f2(b)

d = f3(c)

etc.


In [8]:
# Determine the number of input dimensions (features) of each datapoint
# and use that value to create a tf.keras.Input object, giving it the variable
# name "inputs".
# Also, select a dimension (<=5) for the embedding output give it the variable
# name "embedding_dim".
embedding_dim = 5
inputs = Input(shape=(30,), name="breast_encoder")

In [9]:
assert inputs.shape[1] == X_train.shape[1]
assert isinstance(embedding_dim, int)
assert embedding_dim >0
assert embedding_dim <= 5

In [10]:
# Chain our Input layer to two subsequent Dense layers, the first with 10 neurons (units),
# the second with "embedding_dim" neurons. Name the final output "encoded".
# Use ReLU as the activation function for the first Dense layer and do not set an
# activation for the final layer.

x = Dense(10, activation="relu")(inputs)
encoded = Dense(embedding_dim, activation="relu")(x)

In [11]:
testM = Model(inputs, encoded)
assert len(testM.layers) == 3
assert encoded.shape[1] == embedding_dim

In [12]:
# Chain two additional dense layers, the first with 10 neurons and the
# second (final layer) with the same number of neurons as your input (number
# of features).
# Set the output of the final layer equal to "decoded".

x = Dense(10, activation="relu")(encoded)
decoded = Dense(30)(x)

In [13]:
testM = Model(inputs, decoded)
print(len(testM.layers))
assert len(testM.layers) == 5
assert decoded.shape[1] == 30

5


In [14]:
# Create the autoencoder
autoencoder = Model(inputs,decoded)

# Create the encoder which takes the same inputs but stops at the encoded layers
encoder = Model(inputs, encoded)

# Create the decoder which starts at the encoded output and uses the remaining layers
encoded_embedding = Input(shape=(embedding_dim,))

decoder_layer2 = autoencoder.layers[-2]
decoder_layer3 = autoencoder.layers[-1]

decoder_out = decoder_layer3(decoder_layer2(encoded_embedding))

decoder = Model(encoded_embedding, decoder_out)

In [15]:
# Compile the model with 
autoencoder.compile(optimizer="adam", loss="mse")
autoencoder.fit(X_train, X_train, epochs=1000)

Train on 426 samples
Epoch 1/1000
426/426 [==============================] - 1s 2ms/sample - loss: 56845.9929
Epoch 2/1000
426/426 [==============================] - 0s 81us/sample - loss: 53839.3297
Epoch 3/1000
426/426 [==============================] - 0s 82us/sample - loss: 50915.1612
Epoch 4/1000
426/426 [==============================] - 0s 85us/sample - loss: 46727.5259
Epoch 5/1000
426/426 [==============================] - 0s 86us/sample - loss: 40695.9452
Epoch 6/1000
426/426 [==============================] - 0s 82us/sample - loss: 32458.7166
Epoch 7/1000
426/426 [==============================] - 0s 87us/sample - loss: 23305.9288
Epoch 8/1000
426/426 [==============================] - 0s 85us/sample - loss: 13833.6625
Epoch 9/1000
426/426 [==============================] - 0s 93us/sample - loss: 6467.2158
Epoch 10/1000
426/426 [==============================] - 0s 83us/sample - loss: 2463.1522
Epoch 11/1000
426/426 [==============================] - 0s 85us/sample - loss: 9

426/426 [==============================] - 0s 80us/sample - loss: 115.9012
Epoch 93/1000
426/426 [==============================] - 0s 89us/sample - loss: 112.4104
Epoch 94/1000
426/426 [==============================] - 0s 82us/sample - loss: 110.6014
Epoch 95/1000
426/426 [==============================] - 0s 86us/sample - loss: 111.1014
Epoch 96/1000
426/426 [==============================] - 0s 96us/sample - loss: 109.0370
Epoch 97/1000
426/426 [==============================] - 0s 85us/sample - loss: 104.0790
Epoch 98/1000
426/426 [==============================] - 0s 82us/sample - loss: 102.3877
Epoch 99/1000
426/426 [==============================] - 0s 88us/sample - loss: 99.1047
Epoch 100/1000
426/426 [==============================] - 0s 89us/sample - loss: 100.6117
Epoch 101/1000
426/426 [==============================] - 0s 94us/sample - loss: 96.3798
Epoch 102/1000
426/426 [==============================] - 0s 88us/sample - loss: 94.4536
Epoch 103/1000
426/426 [===========

426/426 [==============================] - 0s 93us/sample - loss: 72.6029
Epoch 185/1000
426/426 [==============================] - 0s 85us/sample - loss: 73.1188
Epoch 186/1000
426/426 [==============================] - 0s 82us/sample - loss: 72.3285
Epoch 187/1000
426/426 [==============================] - 0s 90us/sample - loss: 75.8853
Epoch 188/1000
426/426 [==============================] - 0s 83us/sample - loss: 72.8492
Epoch 189/1000
426/426 [==============================] - 0s 81us/sample - loss: 74.6276
Epoch 190/1000
426/426 [==============================] - 0s 87us/sample - loss: 73.1544
Epoch 191/1000
426/426 [==============================] - 0s 89us/sample - loss: 72.3125
Epoch 192/1000
426/426 [==============================] - 0s 87us/sample - loss: 73.6937
Epoch 193/1000
426/426 [==============================] - 0s 76us/sample - loss: 72.1065
Epoch 194/1000
426/426 [==============================] - 0s 92us/sample - loss: 73.3166
Epoch 195/1000
426/426 [============

426/426 [==============================] - 0s 89us/sample - loss: 68.7323
Epoch 277/1000
426/426 [==============================] - 0s 89us/sample - loss: 70.5117
Epoch 278/1000
426/426 [==============================] - 0s 85us/sample - loss: 71.6521
Epoch 279/1000
426/426 [==============================] - 0s 88us/sample - loss: 71.7843
Epoch 280/1000
426/426 [==============================] - 0s 94us/sample - loss: 71.5247
Epoch 281/1000
426/426 [==============================] - 0s 92us/sample - loss: 70.1295
Epoch 282/1000
426/426 [==============================] - 0s 81us/sample - loss: 70.4966
Epoch 283/1000
426/426 [==============================] - 0s 88us/sample - loss: 70.7447
Epoch 284/1000
426/426 [==============================] - 0s 89us/sample - loss: 71.4838
Epoch 285/1000
426/426 [==============================] - 0s 86us/sample - loss: 68.8473
Epoch 286/1000
426/426 [==============================] - 0s 85us/sample - loss: 70.8859
Epoch 287/1000
426/426 [============

426/426 [==============================] - 0s 96us/sample - loss: 66.9829
Epoch 369/1000
426/426 [==============================] - 0s 95us/sample - loss: 66.5621
Epoch 370/1000
426/426 [==============================] - 0s 87us/sample - loss: 71.1905
Epoch 371/1000
426/426 [==============================] - 0s 83us/sample - loss: 68.0162
Epoch 372/1000
426/426 [==============================] - 0s 85us/sample - loss: 65.8064
Epoch 373/1000
426/426 [==============================] - 0s 88us/sample - loss: 66.1651
Epoch 374/1000
426/426 [==============================] - 0s 89us/sample - loss: 65.2520
Epoch 375/1000
426/426 [==============================] - 0s 92us/sample - loss: 66.4797
Epoch 376/1000
426/426 [==============================] - 0s 93us/sample - loss: 66.8230
Epoch 377/1000
426/426 [==============================] - 0s 89us/sample - loss: 66.6303
Epoch 378/1000
426/426 [==============================] - 0s 92us/sample - loss: 65.3749
Epoch 379/1000
426/426 [============

...

426/426 [==============================] - 0s 87us/sample - loss: 47.7927
Epoch 553/1000
426/426 [==============================] - 0s 85us/sample - loss: 48.0451
Epoch 554/1000
426/426 [==============================] - 0s 92us/sample - loss: 47.8191
Epoch 555/1000
426/426 [==============================] - 0s 88us/sample - loss: 48.7637
Epoch 556/1000
426/426 [==============================] - 0s 85us/sample - loss: 54.0640
Epoch 557/1000
426/426 [==============================] - 0s 92us/sample - loss: 49.6090
Epoch 558/1000
426/426 [==============================] - 0s 89us/sample - loss: 51.2969
Epoch 559/1000
426/426 [==============================] - 0s 94us/sample - loss: 50.0273
Epoch 560/1000
426/426 [==============================] - 0s 88us/sample - loss: 53.0523
Epoch 561/1000
426/426 [==============================] - 0s 82us/sample - loss: 47.4412
Epoch 562/1000
426/426 [==============================] - 0s 89us/sample - loss: 46.2741
Epoch 563/1000
426/426 [============

426/426 [==============================] - 0s 83us/sample - loss: 35.6338
Epoch 645/1000
426/426 [==============================] - 0s 92us/sample - loss: 35.8954
Epoch 646/1000
426/426 [==============================] - 0s 85us/sample - loss: 35.0553
Epoch 647/1000
426/426 [==============================] - 0s 88us/sample - loss: 34.5310
Epoch 648/1000
426/426 [==============================] - 0s 89us/sample - loss: 34.0945
Epoch 649/1000
426/426 [==============================] - 0s 92us/sample - loss: 34.8572
Epoch 650/1000
426/426 [==============================] - 0s 87us/sample - loss: 33.9180
Epoch 651/1000
426/426 [==============================] - 0s 87us/sample - loss: 34.7142
Epoch 652/1000
426/426 [==============================] - 0s 93us/sample - loss: 34.1784
Epoch 653/1000
426/426 [==============================] - 0s 85us/sample - loss: 34.9065
Epoch 654/1000
426/426 [==============================] - 0s 89us/sample - loss: 34.2256
Epoch 655/1000
426/426 [============

426/426 [==============================] - 0s 86us/sample - loss: 27.6083
Epoch 737/1000
426/426 [==============================] - 0s 100us/sample - loss: 27.6798
Epoch 738/1000
426/426 [==============================] - 0s 99us/sample - loss: 27.9559
Epoch 739/1000
426/426 [==============================] - 0s 94us/sample - loss: 27.9773
Epoch 740/1000
426/426 [==============================] - 0s 100us/sample - loss: 27.5772
Epoch 741/1000
426/426 [==============================] - 0s 96us/sample - loss: 27.8517
Epoch 742/1000
426/426 [==============================] - 0s 92us/sample - loss: 27.5850
Epoch 743/1000
426/426 [==============================] - 0s 100us/sample - loss: 27.4289
Epoch 744/1000
426/426 [==============================] - 0s 92us/sample - loss: 27.5442
Epoch 745/1000
426/426 [==============================] - 0s 90us/sample - loss: 27.6442
Epoch 746/1000
426/426 [==============================] - 0s 93us/sample - loss: 27.5960
Epoch 747/1000
426/426 [=========

426/426 [==============================] - 0s 94us/sample - loss: 24.8112
Epoch 829/1000
426/426 [==============================] - 0s 93us/sample - loss: 23.9195
Epoch 830/1000
426/426 [==============================] - 0s 87us/sample - loss: 23.7187
Epoch 831/1000
426/426 [==============================] - 0s 88us/sample - loss: 23.9182
Epoch 832/1000
426/426 [==============================] - 0s 88us/sample - loss: 25.0393
Epoch 833/1000
426/426 [==============================] - 0s 90us/sample - loss: 24.3295
Epoch 834/1000
426/426 [==============================] - 0s 88us/sample - loss: 24.3100
Epoch 835/1000
426/426 [==============================] - 0s 93us/sample - loss: 24.7867
Epoch 836/1000
426/426 [==============================] - 0s 87us/sample - loss: 24.2323
Epoch 837/1000
426/426 [==============================] - 0s 82us/sample - loss: 23.5672
Epoch 838/1000
426/426 [==============================] - 0s 114us/sample - loss: 24.0700
Epoch 839/1000
426/426 [===========

426/426 [==============================] - 0s 87us/sample - loss: 21.5230
Epoch 921/1000
426/426 [==============================] - 0s 87us/sample - loss: 20.7745
Epoch 922/1000
426/426 [==============================] - 0s 95us/sample - loss: 20.5183
Epoch 923/1000
426/426 [==============================] - 0s 88us/sample - loss: 21.0846
Epoch 924/1000
426/426 [==============================] - 0s 92us/sample - loss: 22.0287
Epoch 925/1000
426/426 [==============================] - 0s 101us/sample - loss: 21.9833
Epoch 926/1000
426/426 [==============================] - 0s 96us/sample - loss: 20.8362
Epoch 927/1000
426/426 [==============================] - 0s 92us/sample - loss: 20.6580
Epoch 928/1000
426/426 [==============================] - 0s 98us/sample - loss: 20.4665
Epoch 929/1000
426/426 [==============================] - 0s 93us/sample - loss: 20.8568
Epoch 930/1000
426/426 [==============================] - 0s 94us/sample - loss: 22.2151
Epoch 931/1000
426/426 [===========

In [16]:
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
breast_encoder (InputLayer)  [(None, 30)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                310       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
dense_3 (Dense)              (None, 30)                330       
Total params: 755
Trainable params: 755
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Calculate the embedding using the encoder model
X_train_embed = encoder.predict(X_train)
X_test_embed = encoder.predict(X_test)

In [18]:
# Now, train two LinearSVC models.
# Fit the first model on the original data and name the model "base_model".
# Fit the second model on the autoencoder-embedded data (X_train_embed) and
# name the model "embed_model".

base_model = LinearSVC(random_state=0, tol=1e-5).fit(X_train, y_train)
embed_model = LinearSVC(random_state=0, tol=1e-5).fit(X_train_embed, y_train)

C:\Users\Carson\Anaconda3\envs\NN\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Carson\Anaconda3\envs\NN\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [19]:
assert base_model
assert isinstance(base_model, LinearSVC)
assert base_model.coef_.shape[1] == 30
assert embed_model
assert isinstance(embed_model, LinearSVC)
assert embed_model.coef_.shape[1] == embedding_dim

In [20]:
print(f"The base SVM classifier scores {base_model.score(X_test, y_test)}.")
print(f"The autoencoder embedding SVM classifier scores {embed_model.score(X_test_embed, y_test)}.")

The base SVM classifier scores 0.9020979020979021.
The autoencoder embedding SVM classifier scores 0.7552447552447552.


Was the test set score of the embedded data model better or worse than that of the original data model?

Ask youself why it might be better or worse. 

 - What happens when you change the activation function(s) in the autoencoder?
 - What happens when you change the embedding_dim to be larger or smaller?
 - Is it sufficient to just use LinearSVC with the default parameters to make any of these conclusions?

## Feedback

In [21]:
def feedback():
    """Provide feedback on the contents of this exercise
    
    Returns:
        string
    """
    return "Would have been good experience to have the decoding process as a student task as it was already done and seems like an important part of the process we didn't get to practice on."

feedback()

"Would have been good experience to have the decoding process as a student task as it was already done and seems like an important part of the process we didn't get to practice on."